# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">FINETUNE EMBEDDING</h1></span>

In [1]:
!pip install -qU "sentence-transformers>=2.2.2" peft accelerate transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 2) Environment - must set BEFORE importing torch/transformers heavy modules
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# optionally set accelerate config to avoid HF warnings:
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

# 3) Imports
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, models, losses, evaluation
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, set_peft_model_state_dict
from peft import PeftModel, PeftConfig
from transformers import AutoConfig, AutoTokenizer, AutoModel
from tqdm import tqdm
from peft import TaskType

import wandb
wandb.login(key="9ead951d8a06eac444a5dcd4b9af1a9b89cc6cc2")

2025-11-16 01:08:18.624510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763255298.881455      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763255298.950461      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [3]:
# ---------------- USER CONFIG ----------------
MODEL_NAME = "AITeamVN/Vietnamese_Embedding"   # base HF model (sentence-transformers wrapper will use it)
OUTPUT_DIR = "fine_tuned_lora_bgem3"
LORA_DIR = "lora_adapter"
BATCH_SIZE = 8            # try 8; lower to 4 if OOM
EPOCHS = 3
LR = 2e-4                 # LoRA can use higher lr than full FT
LORA_R = 8                # rank
LORA_ALPHA = 32
TARGET_MODULES = ["q_proj","v_proj","k_proj","o_proj","query","key","value","dense"]  # try broad set
SEED = 42

# reproducible
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# 4) Read dataset (adjust paths if needed)
data_dir = "/kaggle/input/vietnamhistory-dataset/"  # change if needed
train_df = pd.read_csv(os.path.join(data_dir, "train_pairs.csv"))
val_df   = pd.read_csv(os.path.join(data_dir, "valid_pairs.csv"))
test_df  = pd.read_csv(os.path.join(data_dir, "test_pairs.csv"))

print("Train/Val/Test sizes:", len(train_df), len(val_df), len(test_df))

Train/Val/Test sizes: 14992 3216 3214


In [4]:
# 5) Prepare sentence-transformers modules manually so we can swap HF model with LoRA later
# We will first build a SentenceTransformer with a Transformer module that loads the HF model
hf_transformer = models.Transformer(MODEL_NAME, max_seq_length=512)   # keep original seq len
pooling = models.Pooling(hf_transformer.get_word_embedding_dimension(), pooling_mode_cls_token=False,
                         pooling_mode_mean_tokens=True, pooling_mode_max_tokens=False)
st_modules = [hf_transformer, pooling]
st_model = SentenceTransformer(modules=st_modules)

# 6) Prepare training examples:
# For MultipleNegativesRankingLoss we need only positive pairs (label==1.0)
train_pos = train_df[train_df['label'] == 1.0]
train_examples = [InputExample(texts=[q, c]) for q, c in zip(train_pos['question'].astype(str), train_pos['context'].astype(str))]
print("Positive train examples:", len(train_examples))

# For evaluator (we keep pairs with labels 0/1 to compute Pearson/Spearman)
val_examples = [InputExample(texts=[q, c], label=float(l)) for q, c, l in zip(val_df['question'].astype(str), val_df['context'].astype(str), val_df['label'])]
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name="val-eval")

# 7) Attach LoRA to the underlying HuggingFace model inside our transformer
# get access to HF model
hf_model = st_model._first_module().auto_model  # huggingface model inside sentence-transformers Transformer module
tokenizer = st_model._first_module().tokenizer

# Prepare the HF model for PEFT (optional for k-bit training; safe to call)
# (we do not use k-bit here but prepare_model_for_kbit_training is safe; skip if undesired)
try:
    hf_model = prepare_model_for_kbit_training(hf_model)
except Exception:
    # not critical, continue
    pass

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    bias="none",
)

# Wrap HF model with LoRA
peft_model = get_peft_model(hf_model, lora_config)
# Replace the transformer auto_model inside st_model by the peft-wrapped model
st_model._first_module().auto_model = peft_model

# Move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
st_model.to(device)

# Print number of trainable params (should be small)
def count_trainable_params(model):
    total=0; trainable=0
    for n,p in model.named_parameters():
        total += p.numel()
        if p.requires_grad:
            trainable += p.numel()
    return total, trainable

tot, trainable = count_trainable_params(st_model._first_module().auto_model)
print(f"Total params (hf): {tot:,}, trainable (after LoRA): {trainable:,}")

# 8) DataLoader (use positive examples for MultipleNegativesRankingLoss)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)

# 9) Loss: MultipleNegativesRankingLoss (in-batch negatives)
train_loss = losses.MultipleNegativesRankingLoss(st_model)

# 10) Warmup steps
total_steps = (len(train_dataloader) * EPOCHS)
warmup_steps = max(1, int(0.1 * total_steps))
print("Total steps:", total_steps, "Warmup steps:", warmup_steps)

# 11) Fit
os.makedirs(OUTPUT_DIR, exist_ok=True)

st_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=EPOCHS,
    evaluation_steps=200,
    warmup_steps=warmup_steps,
    optimizer_params={"lr": LR},
    output_path=OUTPUT_DIR,
    use_amp=True
)

# 12) Save LoRA adapter separately (PEFT style)
# The peft model is stored inside st_model._first_module().auto_model
peft_backend = st_model._first_module().auto_model
# Save PEFT adapter weights
peft_backend.save_pretrained(LORA_DIR)
print("Saved LoRA adapter to", LORA_DIR)

# 13) Save full sentence-transformer (optionally; will be heavy)
st_model.save(os.path.join(OUTPUT_DIR, "sentence_transformer_full"))

# 14) Example: how to load base model + adapter later for inference
# (a) load base sentence-transformer; (b) load adapter into hf model; (c) rebuild sentence-transformer wrapper
from peft import PeftModel, PeftConfig

# Load base modules
base_transformer = models.Transformer(MODEL_NAME, max_seq_length=512)
base_pooling = models.Pooling(base_transformer.get_word_embedding_dimension())
base_st = SentenceTransformer(modules=[base_transformer, base_pooling])

# Pull HF model and load adapter
hf_base = base_st._first_module().auto_model
hf_base = PeftModel.from_pretrained(hf_base, LORA_DIR)  # attach adapter
base_st._first_module().auto_model = hf_base
base_st.to(device)

# Quick eval on some sample
sample_q = ["Ai là chủ tịch nước?"]
sample_c = ["Chủ tịch nước hiện nay là ..."]
q_emb = base_st.encode(sample_q, convert_to_tensor=True)
c_emb = base_st.encode(sample_c, convert_to_tensor=True)
cos = torch.nn.functional.cosine_similarity(q_emb, c_emb)
print("Sample similarity:", cos.item())

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

...

Positive train examples: 7496
Total params (hf): 571,310,080, trainable (after LoRA): 3,555,328
Total steps: 2811 Warmup steps: 281


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251116_010857-qh6hucdc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dry-blaze-15
wandb: ⭐️ View project at https://wandb.ai/21522798-uit/sentence-transformers
wandb: 🚀 View run at https://wandb.ai/21522798-uit/sentence-transformers/runs/qh6hucdc


Step,Training Loss,Validation Loss,Val-eval Pearson Cosine,Val-eval Spearman Cosine
200,No log,No log,0.896899,0.857219
400,No log,No log,0.908904,0.859784
600,0.152300,No log,0.916292,0.860888
800,0.152300,No log,0.917038,0.861217
937,0.152300,No log,0.921147,0.861415
1000,0.058900,No log,0.918947,0.860930
1200,0.058900,No log,0.920265,0.861339
1400,0.058900,No log,0.922199,0.861038
1600,0.034800,No log,0.922872,0.861333
1800,0.034800,No log,0.926453,0.862275


Saved LoRA adapter to lora_adapter
Sample similarity: 0.7560317516326904


In [5]:
# 15) Evaluate P@1 on test set (full)
def evaluate_p_at_k_full(model, df, topk=1):
    examples = [InputExample(texts=[q,c]) for q,c in zip(df['question'].astype(str), df['context'].astype(str))]
    questions = [ex.texts[0] for ex in examples]
    contexts  = [ex.texts[1] for ex in examples]
    q_emb = model.encode(questions, convert_to_tensor=True, normalize_embeddings=True)
    c_emb = model.encode(contexts, convert_to_tensor=True, normalize_embeddings=True)
    scores = torch.mm(q_emb, c_emb.T)
    correct = 0
    for i in range(len(questions)):
        topk_idx = torch.topk(scores[i], k=topk).indices.cpu().tolist()
        retrieved = [contexts[j] for j in topk_idx]
        golds = df[df['question']==questions[i]]['context'].tolist()
        if any(r in golds for r in retrieved):
            correct += 1
    return correct / len(questions)

print("Evaluating base+adapter on test set...")
p1 = evaluate_p_at_k_full(base_st, test_df, topk=1)
print("P@1 (base+LoRA):", p1)

Evaluating base+adapter on test set...
P@1 (base+LoRA): 0.7542003733665215
